In [1]:
import pybullet as p
import pybullet_data
import time
import math
import random
import csv
import xlsxwriter

In [7]:
p.disconnect()


error: Not connected to physics server.

In [2]:
def applied_speed(speed):
    return (speed+2.1536)/0.1454


def zumoVel_to_bullet(zumoVel):
    return (28.0/400.0)*zumoVel

In [82]:
header = ["Time", "Error"]

# Create a workbook and add a worksheet.
workbook = xlsxwriter.Workbook('test-rise.xlsx')

#Sheet names
name = "PD"

sheets = {}



In [3]:
#file = open("PID.csv", "w", newline="")
#writer = csv.writer(file, delimiter=";")
#writer.writerow(header)

#start the simulator in either GUI or DIRECT mode
physicsclient = p.connect(p.GUI) #change to "p.GUI" to visualize the simulation

p.configureDebugVisualizer(p.COV_ENABLE_WIREFRAME, 0)
p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)

#Set path to pybullet data, which is used to load the standard plane
p.setAdditionalSearchPath(pybullet_data.getDataPath())

#Load the plane
plainID = p.loadURDF("plane.urdf")

#Load the simplified zumo
robot = p.loadURDF("SimpleZumo.urdf",
                    [0,0,0.01])

#Set gravity
p.setGravity(0,0,-10)
p.setRealTimeSimulation(0)
p.resetDebugVisualizerCamera(cameraDistance=0.8, cameraYaw=0, cameraPitch=-65, cameraTargetPosition=[0,0,0])

In [5]:
print(p.getDynamicsInfo(robot, plainID))
print(p.getConnectionInfo(physicsclient)["connectionMethod"])


(0.12999123516761424, 0.5, (4.3828996071739895e-05, 2.666948502831625e-05, 2.6712530825646186e-05), (-0.007565715755765867, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0), 0.0, 0.0, 0.0, -1.0, -1.0, 2, 0.001)
1


In [4]:
p.changeDynamics(robot, plainID, 0.6, 0.5, 0, 0.8)

In [5]:
def calculate_fitness(sq_error, sq_output):
    return 1/(100*sq_error + sq_output + 0.00000000001)

def select_next_gen(list_of_gains, list_of_fitness, total_fitness, advancing_num = 40):
    next_gen_gains = []
    probs = []
    cum_prob = [] 
    prob_sum = 0

    for i in range(len(list_of_fitness)):
        prob = list_of_fitness[i]/total_fitness
        probs.append(prob)

        prob_sum += prob
        cum_prob.append(prob_sum)

    #We need to save the idx of the best chromosome to make it proceed to next generation
    best_idx = probs.index(max(probs))
    next_gen_gains.append(list_of_gains[best_idx])
    
    for i in range(advancing_num-1):
        r = random.random()
        n = 0

        while True: #for n in range(len(cum_prob)):
            if r < cum_prob[n]:
                if list_of_gains[n] not in next_gen_gains:
                    next_gen_gains.append(list_of_gains[n])
                    break
                else:
                    r = random.random()
                    n = -1

            n += 1
                
    return next_gen_gains


def crossover_and_mutate(gains, cross_val = 0.8, num_cross = 60, num_mutation = 39):
    next_generation_gains = []
    
    #Append the best from previous generation
    next_generation_gains.append(gains[0])

    n = 0

    while True: #for i in range(num_cross/2):
        parent1 = gains[random.randint(1, len(gains)-1)]
        parent2 = gains[random.randint(1, len(gains)-1)]

        #Make sure that the mates are different
        while parent2 == parent1:
            parent2 = gains[random.randint(1, len(gains)-1)]

        child1 = [cross_val*parent1[0]+(1-cross_val)*parent2[0], cross_val*parent1[1]+(1-cross_val)*parent2[1], cross_val*parent1[2]+(1-cross_val)*parent2[2]]
        child2 = [cross_val*parent2[0]+(1-cross_val)*parent1[0], cross_val*parent2[1]+(1-cross_val)*parent1[1], cross_val*parent2[2]+(1-cross_val)*parent1[2]]

        if child1 not in next_generation_gains:
            next_generation_gains.append(child1)
            next_generation_gains.append(child2)
            n += 1
        
        if n == num_cross/2:
            break

    for i in range(num_mutation):
        mutation = gains[random.randint(1, len(gains)-1)]

        mutation[0] = mutation[0] * random.uniform(0.75, 1.25)
        mutation[1] = mutation[1] * random.uniform(0.75, 1.25)
        mutation[2] = mutation[1] * random.uniform(0.75, 1.25)

        next_generation_gains.append(mutation)

    return next_generation_gains
    

    


In [6]:
dist = 200

generations = 5
num_individuals = 100
num_advance = 40
num_cross = 60
num_mut = 39
error_increased = False

gains = []
best_gains = []
best_fitness = []
fitness = []
fitness_sum = 0

gen_error = 0
gen_fitness = []

for i in range(num_individuals):
    gains.append([round(random.uniform(0, 300),2), round(random.uniform(0, 300)), round(random.uniform(0, 300))])

for gen in range(generations): 
    for n in range(num_individuals):
        speedRight = 30.0
        speedLeft = 30.0

        VelRight = zumoVel_to_bullet(applied_speed(speedRight))
        VelLeft = zumoVel_to_bullet(applied_speed(speedLeft))

        #kc = gains[n][0]
        #kd = gains[n][1]
        #ki = gains[n][2]

        kc = gains[n][0]
        kd = kc*gains[n][1]
        ki = kc*(1/(gains[n][2]+0.000000001))

            
        lastError = 0
        integral_error = 0 
        cum_sq_output = 0  
        cum_sq_error = 0   

        angle = p.getQuaternionFromEuler([0, 0, 0.06])
        p.resetBasePositionAndOrientation(robot, [0,0,0.01], angle) #[0,0,0,1])
        p.resetDebugVisualizerCamera(cameraDistance=0.8, cameraYaw=0, cameraPitch=-75, cameraTargetPosition=[0,0,0])

        p.setJointMotorControl2(robot, 0, p.VELOCITY_CONTROL, targetVelocity = 0)
        p.setJointMotorControl2(robot, 1, p.VELOCITY_CONTROL, targetVelocity = 0)
        p.setJointMotorControl2(robot, 2, p.VELOCITY_CONTROL, targetVelocity = 0)
        p.setJointMotorControl2(robot, 3, p.VELOCITY_CONTROL, targetVelocity = 0)

        for i in range(100):
            p.stepSimulation()

        p.setJointMotorControl2(robot, 0, p.VELOCITY_CONTROL, targetVelocity = VelLeft)
        p.setJointMotorControl2(robot, 1, p.VELOCITY_CONTROL, targetVelocity = VelLeft)
        p.setJointMotorControl2(robot, 2, p.VELOCITY_CONTROL, targetVelocity = VelRight)
        p.setJointMotorControl2(robot, 3, p.VELOCITY_CONTROL, targetVelocity = VelRight)

        i = 0

        while True:
            p.resetDebugVisualizerCamera(cameraDistance=0.8, cameraYaw=0, cameraPitch=-75, cameraTargetPosition=[0,i/(450+(400-speedRight)),0])
            #i%4 svarer til at den opdaterer med samme interval som zumoen
            #hvert step (p.stepSimulation) svarer til 1/240 s. 4*1/240 = 16ms, hvilket svarer til zumoens korrektionsinterval
            if(i % 4 == 0):
                pos, orient = p.getBasePositionAndOrientation(robot)
                angle = p.getEulerFromQuaternion(orient)
                error = 0 - angle[2] * 180.0/math.pi

                cum_sq_error += error**2
                gen_error += error
                integral_error += error
            
                speedDiff = float(kc*error + ki * integral_error + kd*(error-lastError))
                cum_sq_output = speedDiff**2
                
                #speedDiff = float(kc*error + kd*(error-lastError))
                
                lastError = error

                VelRight -=  zumoVel_to_bullet(speedDiff)
                VelLeft += zumoVel_to_bullet(speedDiff)

                VelRight = zumoVel_to_bullet(70) if VelRight < zumoVel_to_bullet(70) else (zumoVel_to_bullet(400) if VelRight > zumoVel_to_bullet(400) else VelRight)
                VelLeft = zumoVel_to_bullet(70) if VelLeft < zumoVel_to_bullet(70) else (zumoVel_to_bullet(400) if VelLeft > zumoVel_to_bullet(400) else VelLeft)

                p.setJointMotorControl2(robot, 0, p.VELOCITY_CONTROL, targetVelocity = VelLeft)
                p.setJointMotorControl2(robot, 1, p.VELOCITY_CONTROL, targetVelocity = VelLeft)
                p.setJointMotorControl2(robot, 2, p.VELOCITY_CONTROL, targetVelocity = VelRight)
                p.setJointMotorControl2(robot, 3, p.VELOCITY_CONTROL, targetVelocity = VelRight)

            p.stepSimulation()
            pos, orient = p.getBasePositionAndOrientation(robot)
            

            if pos[1] >= (dist/100) or i > 2000:
                fit = calculate_fitness(cum_sq_error, cum_sq_output)
                fitness.append(fit)
                fitness_sum += fit

                break
            elif pos[1] >= (dist/200) and error_increased == False:
                angle = p.getQuaternionFromEuler([0, 0, -0.06])
                p.resetBasePositionAndOrientation(robot, pos, angle)
                error_increased = True
                i += 1
            else:
                i += 1


    best_fitness.append(max(fitness))
    tmp_gains = select_next_gen(gains, fitness, fitness_sum, num_advance)
    gains = crossover_and_mutate(tmp_gains, num_cross=num_cross, num_mutation=num_mut)
    best_gains.append(gains[0])
    gen_fitness.append(fitness_sum/num_individuals)
    gen_error = 0
    error_increased = False

    print(f"Generation {gen}")
    print(f"Best gain: {gains[0]}  Avg. fitness: {gen_fitness[gen]}  PopSize: {len(gains)}\n")

    fitness_sum = 0
    fitness = []

        

error: GetBasePositionAndOrientation failed.

In [18]:
sheets["Data"] = workbook.add_worksheet("Data")
errors = []

col = 1
for gen in range(len(best_gains)): 
    row = 0
    #sheets[name + str(gen)] = workbook.add_worksheet(str(name) + str(gen))
    #sheets[name + str(gen)].write(row, 0, header[0])
    #sheets[name + str(gen)].write(row, 1, header[1])
    sheets["Data"].write(row, 0, header[0])
    sheets["Data"].write(row, col, f"Error{gen}")
    row += 1

    speedRight = 30.0
    speedLeft = 30.0

    VelRight = zumoVel_to_bullet(applied_speed(speedRight))
    VelLeft = zumoVel_to_bullet(applied_speed(speedLeft))

    #kc = round(best_gains[gen][0])
    #kd = round(best_gains[gen][1])
    #ki = round(best_gains[gen][2])

    kc = round(best_gains[gen][0],2)
    kd = round(kc*best_gains[gen][1],2)
    ki = round(kc*(1/best_gains[gen][2]),2)
    

    lastError = 0
    error_total = 0 
    integral_error = error
    error_increased = False  
    riseTime = 0
    riseCount = 0   

    #angle = p.getQuaternionFromEuler([0, 0, 0.15])
    angle = p.getQuaternionFromEuler([0, 0, 0.06])
    p.resetBasePositionAndOrientation(robot, [0,0,0.01], angle) #[0,0,0,1])
    p.resetDebugVisualizerCamera(cameraDistance=0.8, cameraYaw=0, cameraPitch=-75, cameraTargetPosition=[0,0,0])

    p.setJointMotorControl2(robot, 0, p.VELOCITY_CONTROL, targetVelocity = 0)
    p.setJointMotorControl2(robot, 1, p.VELOCITY_CONTROL, targetVelocity = 0)
    p.setJointMotorControl2(robot, 2, p.VELOCITY_CONTROL, targetVelocity = 0)
    p.setJointMotorControl2(robot, 3, p.VELOCITY_CONTROL, targetVelocity = 0)

    for i in range(100):
        p.stepSimulation()

    p.setJointMotorControl2(robot, 0, p.VELOCITY_CONTROL, targetVelocity = VelLeft)
    p.setJointMotorControl2(robot, 1, p.VELOCITY_CONTROL, targetVelocity = VelLeft)
    p.setJointMotorControl2(robot, 2, p.VELOCITY_CONTROL, targetVelocity = VelRight)
    p.setJointMotorControl2(robot, 3, p.VELOCITY_CONTROL, targetVelocity = VelRight)

    i = 0

    while True:
        p.resetDebugVisualizerCamera(cameraDistance=0.8, cameraYaw=0, cameraPitch=-75, cameraTargetPosition=[0,i/(450+(400-speedRight)),0])
        
        #i%4 svarer til at den opdaterer med samme interval som zumoen
        #hvert step (p.stepSimulation) svarer til 1/240 s. 4*1/240 = 16ms, hvilket svarer til zumoens korrektionsinterval
        if(i % 4 == 0):
            pos, orient = p.getBasePositionAndOrientation(robot)
            angle = p.getEulerFromQuaternion(orient)
            error = 0 - angle[2] * 180.0/math.pi

            #sheets[name + str(gen)].write(row, 0, i/240)
            #sheets[name + str(gen)].write(row, 1, error)
            sheets["Data"].write(row, 0, i/240)
            sheets["Data"].write(row, col, error)
            row += 1

            error_total += abs(error)
            integral_error += error
            
            speedDiff = float(kc*error + ki * integral_error + kd*(error-lastError))
            
            lastError = error

            VelRight -=  zumoVel_to_bullet(speedDiff)
            VelLeft += zumoVel_to_bullet(speedDiff)

            VelRight = zumoVel_to_bullet(70) if VelRight < zumoVel_to_bullet(70) else (zumoVel_to_bullet(400) if VelRight > zumoVel_to_bullet(400) else VelRight)
            VelLeft = zumoVel_to_bullet(70) if VelLeft < zumoVel_to_bullet(70) else (zumoVel_to_bullet(400) if VelLeft > zumoVel_to_bullet(400) else VelLeft)

            p.setJointMotorControl2(robot, 0, p.VELOCITY_CONTROL, targetVelocity = VelLeft)
            p.setJointMotorControl2(robot, 1, p.VELOCITY_CONTROL, targetVelocity = VelLeft)
            p.setJointMotorControl2(robot, 2, p.VELOCITY_CONTROL, targetVelocity = VelRight)
            p.setJointMotorControl2(robot, 3, p.VELOCITY_CONTROL, targetVelocity = VelRight)

        p.stepSimulation()
        pos, orient = p.getBasePositionAndOrientation(robot)
        

        if pos[1] >= (dist/100) or i > 2000:
            #sheets[name + str(gen)].write(0, 3, f"Kc = {kc}, Kd = {kd}, Ki = {ki}\nTotal Error = {round(error_total,2)}")
            errors.append(round(error_total, 2))
            break
        elif pos[1] >= (dist/200) and error_increased == False:
            angle = p.getQuaternionFromEuler([0, 0, -0.06])
            p.resetBasePositionAndOrientation(robot, pos, angle)
            error_increased = True
        else:
            i += 1
    
    col += 1

In [19]:
sheets["Graphs"] = workbook.add_worksheet("Graphs")

row = 0
graph_error = True

for i in range(len(best_gains)):
    chart = workbook.add_chart({'type': 'scatter',
                             'subtype': 'straight'})

    chart.add_series({
        'name': '',
        'categories': '=Data!$A$2:$A$500',
        'values':     ['Data', 0, i+1, 500, i+1],
        'data_labels': {'series_name': False}
    })

    # Add a chart title 
    chart.set_title({ 'name': f'Kc = {round(best_gains[i][0],2)}, Kd = {round(best_gains[i][1], 2)}, Ki = {round(best_gains[i][2],)}\n Error = {round(errors[i],2)}',
                        'name_font': {'size': 14}})

    # Add x-axis label
    chart.set_x_axis({'name': 'Time [s]', 'major_unit': 1}) 

    chart.set_legend({'none': True})

    # Add y-axis label
    chart.set_y_axis({ 'major_unit': 0.1,
                        'min': -0.4, 'max': 0.4}) #'name': 'Angular Error [degrees]'

    if i%2 == 0: 
        sheets["Graphs"].insert_chart(f'A{row*11+1}', chart, {'x_scale': 0.59, 'y_scale': 0.75,})  
    else:
        sheets["Graphs"].insert_chart(f'E{row*11+1}', chart, {'x_offset': 30, 'x_scale': 0.6, 'y_scale': 0.75,})
        row+=1

row = 0

#Make an chart over the generations' average error
if graph_error:
    sheets["Error-Graph"] = workbook.add_worksheet("Error-Graph")
    for i in range(len(gen_fitness)):
        sheets["Error-Graph"].write(row, 0, i)
        sheets["Error-Graph"].write(row, 1, gen_fitness[i])
        sheets["Error-Graph"].write(row, 2, best_fitness[i])
        row += 1


    chart = workbook.add_chart({'type': 'scatter',
                             'subtype': 'straight'})
    chart2 = workbook.add_chart({'type': 'scatter',
                             'subtype': 'straight'})                      
    chart.add_series({
        'name': '',
        'categories': f'=Error-Graph!$A$1:$A${len(gen_fitness)}',
        'values':     f'=Error-Graph!$B$1:$B${len(gen_fitness)}',
        'data_labels': {'series_name': False}
    })
    chart2.add_series({
        'name': '',
        'categories': f'=Error-Graph!$A$1:$A${len(gen_fitness)}',
        'values':     f'=Error-Graph!$C$1:$C${len(gen_fitness)}',
        'data_labels': {'series_name': False}
    })

    # Add a chart title 
    chart.set_title({ 'name': 'Avg. Generation Fitness',
                        'name_font': {'size': 14}})
    chart2.set_title({ 'name': 'Generation Best Fitness',
                        'name_font': {'size': 14}})

    # Add x-axis label
    chart.set_x_axis({'name': 'Generation'}) 
    # Add x-axis label
    chart2.set_x_axis({'name': 'Generation'}) 

    # Add y-axis label
    chart.set_y_axis({'name': 'Avg. fitness'})

    # Add y-axis label
    chart2.set_y_axis({'name': 'Fitness'})

    chart.set_legend({'none': True})
    chart2.set_legend({'none': True})

    # Insert the chart into the worksheet.
    sheets["Error-Graph"].insert_chart('D5', chart)
    sheets["Error-Graph"].insert_chart('D20', chart2)

In [20]:
workbook.close()
p.disconnect()

In [83]:
workbook.close()

In [303]:
p.disconnect()


In [446]:
best_gains= [[13.5, 8, 9]]
errors = [5]
gen_fitness = [5]
dist=200

In [ ]:
for i in range(700):
    p.stepSimulation()
    time.sleep(1/240)

In [ ]:
iterations = generations

sheets[name + str(iterations+1)] = workbook.add_worksheet(str(name) + str(iterations+1))

row = 0
graph_error = True

for i in range(iterations):
    chart = workbook.add_chart({'type': 'scatter',
                             'subtype': 'straight'})
    #chart = workbook.add_chart({'type': 'line'})

    chart.add_series({
        'name': '',
        'categories': f'={name}{i}!$A$2:$A$500',
        'values':     f'={name}{i}!$B$2:$B$500',
        'data_labels': {'series_name': False}
    })

    # Add a chart title 
    chart.set_title({ 'name': f'={name}{i}!$D$1',
                        'name_font': {'size': 14}})

    # Add x-axis label
    chart.set_x_axis({'name': 'Time [s]', 'major_unit': 0.5}) 

    # Add y-axis label
    chart.set_y_axis({'name': 'Angular Error [degrees]'})

    chart.set_legend({'none': True})

    # Insert the chart into the worksheet.
    sheets[name + str(i)].insert_chart('D5', chart)

    #You can not insert the chart two times and it therefore necessary to creat the chart again
    chart = workbook.add_chart({'type': 'scatter',
                             'subtype': 'straight'})
    #chart = workbook.add_chart({'type': 'line'})

    chart.add_series({
        'name': '',
        'categories': f'={name}{i}!$A$2:$A$500',
        'values':     f'={name}{i}!$B$2:$B$500',
        'data_labels': {'series_name': False}
    })

    # Add a chart title 
    chart.set_title({ 'name': f'={name}{i}!$D$1',
                        'name_font': {'size': 12}})

    # Add x-axis label
    chart.set_x_axis({'name': 'Time [s]', 'major_unit': 1}) 

    # Add y-axis label
    chart.set_y_axis({ 'major_unit': 0.1,
                        'min': -0.2, 'max': 0.2}) #'name': 'Angular Error [degrees]'

    chart.set_legend({'none': True})

    if i%2 == 0: 
        sheets[name + str(iterations+1)].insert_chart(f'A{row*11+1}', chart, {'x_scale': 0.59, 'y_scale': 0.75,})  
    else:
        sheets[name + str(iterations+1)].insert_chart(f'E{row*11+1}', chart, {'x_offset': 30, 'x_scale': 0.6, 'y_scale': 0.75,})
        row+=1

row = 0

#Make an chart over the generations' average error
if graph_error:
    sheets[name + str(iterations+2)] = workbook.add_worksheet(str(name) + str(iterations+2))
    for i in range(len(gen_fitness)):
        sheets[name + str(iterations+2)].write(row, 0, i)
        sheets[name + str(iterations+2)].write(row, 1, gen_fitness[i])
        row += 1


    chart = workbook.add_chart({'type': 'scatter',
                             'subtype': 'straight'})
    chart.add_series({
        'name': '',
        'categories': f'={name}{iterations+2}!$A$1:$A${len(gen_fitness)}',
        'values':     f'={name}{iterations+2}!$B$1:$B${len(gen_fitness)}',
        'data_labels': {'series_name': False}
    })

    # Add a chart title 
    chart.set_title({ 'name': 'Avg. Generation Error',
                        'name_font': {'size': 14}})

    # Add x-axis label
    chart.set_x_axis({'name': 'Generation', 'major_unit': 1}) 

    # Add y-axis label
    chart.set_y_axis({'name': 'Avg. error'})

    chart.set_legend({'none': True})

    # Insert the chart into the worksheet.
    sheets[name + str(iterations+2)].insert_chart('D5', chart)

In [ ]:
pos, orient = p.getBasePositionAndOrientation(robot)

angle = p.getEulerFromQuaternion(orient)
print(pos)
print(angle[2])

In [ ]:
VelRight = -40
VelLeft = -40

kc = 10
kd = 2

lastError = 0

for i in range(100):
    #p.stepSimulation()
    time.sleep(1/240)

p.setJointMotorControl2(robot, 0, p.VELOCITY_CONTROL, targetVelocity = VelLeft, maxVelocity = 100)
p.setJointMotorControl2(robot, 1, p.VELOCITY_CONTROL, targetVelocity = VelLeft, maxVelocity = 100)
p.setJointMotorControl2(robot, 2, p.VELOCITY_CONTROL, targetVelocity = VelRight, maxVelocity = 100)
p.setJointMotorControl2(robot, 3, p.VELOCITY_CONTROL, targetVelocity = VelRight, maxVelocity = 100)

In [ ]:
#lad os få lidt ekstra information omkring joint 7
p.getJointInfo(robot, 2)
